In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(rotation_range= 40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
img = load_img('/content/sample_data/cat with mask.jpg')
x= img_to_array(img)
x= x.reshape((1,)+x.shape)
i=0
for batch in datagen.flow(x, batch_size=1, save_to_dir='/content/sample_data/preview', save_prefix='cat', save_format='jpeg'):
  i +=1
  if(i>20):
    break

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.5.0


In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images, train_labels), (test_images,test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images = train_images.reshape(len(train_images), 28,28,1)
test_images = test_images.reshape(len(test_images), 28,28,1)

In [84]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_unit', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])),
              loss ='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [74]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [85]:
tuner_search = RandomSearch(build_model, objective='val_accuracy',max_trials=5, directory='output', project_name='mnist_fashion')

In [86]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.1054999977350235

Best val_accuracy So Far: 0.909166693687439
Total elapsed time: 00h 03m 03s
INFO:tensorflow:Oracle triggered exit


In [87]:
model = tuner_search.get_best_models(num_models=1)[0]

In [88]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 112)       2912      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        32288     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1486944   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,523,114
Trainable params: 1,523,114
Non-trainable params: 0
_________________________________________________________________


In [89]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1634 - accuracy: 0.9381 - val_loss: 0.2584 - val_accuracy: 0.9078
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1266 - accuracy: 0.9520 - val_loss: 0.3065 - val_accuracy: 0.9067
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0973 - accuracy: 0.9634 - val_loss: 0.3110 - val_accuracy: 0.9158
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0747 - accuracy: 0.9716 - val_loss: 0.3339 - val_accuracy: 0.9158
Epoch 8/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0579 - accuracy: 0.9781 - val_loss: 0.3610 - val_accuracy: 0.9122
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0446 - accuracy: 0.9838 - val_loss: 0.4285 - val_accuracy: 0.9137
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0375 - accuracy: 0.9860 - val_loss: 0.4346 - val_accuracy